In [14]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle

In [15]:
user_based_recommendation = pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/bert_test_data.csv')
validation_test_data=pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/validation_testing_data.csv')
df_news_all=pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/df_news_all.csv')#新聞內容
target_user_id=pd.read_csv('/project/at081-group38/AT081150_AN/193_ER/target_user_id.csv') # 預測者的ID

In [16]:
#讀取目標user_id
target_user_id=target_user_id.iloc[0,:].values[0]

In [17]:
#從validation_test_data中找出user_id的validation 的新聞
validation_data_guid = validation_test_data[validation_test_data.user_id == target_user_id].iloc[0,:].guid

In [18]:
df_news_all['label']=1

In [19]:
#抓取validation_data_guid的新聞
temp = df_news_all[df_news_all.news_guid==validation_data_guid]

In [20]:
#如果不存在裡面的話，就把加入Validation_data
if user_based_recommendation[user_based_recommendation.news_guid==validation_data_guid].empty:
    print('目標不存在候選集合當中')
    user_based_recommendation=user_based_recommendation.append(temp)
else:
    print('目標已存在候選集合當中')

目標已存在候選集合當中


In [21]:
bert_test_data_with_target = user_based_recommendation
bert_test_data_with_target.head(3)

,path,date,news_guid,title,news_content,content_remove_html_tag,label
0,../datasets/news/2018-02-12/9b2e10c5-1eee-4081...,20180212104054,9B2E10C5-1EEE-4081-A3AB-566A99845EF5,《1月營收》出貨迎成長，醫揚/聯合創新高 ...,\n<P>MoneyDJ新聞 2018-02-12 10:40:54 記者 劉莞青 報導</...,MoneyDJ新聞 2018-02-12 10:40:54 記者 劉莞青 報導聯合(4129...,1
1,../datasets/news/2018-02-12/3e3b2e23-2780-4aae...,20180212095805,3E3B2E23-2780-4AAE-9B54-BC01F2E29526,《1月營收》被動元件、擴產各擁題材，多家營收創高 ...,\n<P>MoneyDJ新聞 2018-02-12 09:58:05 記者 林詩茵 報導</...,MoneyDJ新聞 2018-02-12 09:58:05 記者 林詩茵 報導被動元件廠商受...,1
2,../datasets/news/2018-02-12/d9865670-a654-43c7...,20180212132606,D9865670-A654-43C7-B7B2-9CBE04511E20,《1月營收》文創遊戲更改營收認列方式 橘子續創高 ...,\n<P>MoneyDJ新聞 2018-02-12 13:26:06 記者 萬惠雯 報導</...,MoneyDJ新聞 2018-02-12 13:26:06 記者 萬惠雯 報導\n文化創意產...,1


In [22]:
bert_test_data_with_target.to_csv('/project/at081-group38/AT081150_AN/193_ER/bert_test_data_with_target.csv', index = 0)

In [23]:
data_for_bert = []
for i in range(len(user_based_recommendation)):
    data_for_bert.append((i, #guid
                          user_based_recommendation.iloc[i,:]['title'].strip() + ' ' + user_based_recommendation.iloc[i,:]["content_remove_html_tag"],#text_a
                          None, #text_b
                          user_based_recommendation.iloc[i,:].label #label 雖然用不到但還是得給
                         ))

test = data_for_bert

In [24]:
test[0]

(0,
 '《1月營收》出貨迎成長，醫揚/聯合創新高 MoneyDJ新聞 2018-02-12 10:40:54 記者 劉莞青 報導聯合(4129)持續受到旺季帶動，各地市場出貨成長，加上去年從第二季起才併入冠亞生技營收，第一季營收基期仍低，聯合元月營收來到2.12億元、月增15%、年增110%、創下新高，而醫療工業電腦廠商醫揚(6569)也受惠於年底行動護理專案持續出貨，元月營收也創下歷史新高。另外包括邦特(4107)、泰博(4736)元月持續受到客戶出貨成長，也帶動邦特、泰博元月營收年增來到兩成以上。去年因歐洲客戶受到選舉年干擾，專案預算遞延至年底才開始陸續出貨，而受惠於行動護理產品等專案持續於元月出貨，醫揚元月營收來到1.48億元、月增24%、年增46%，也創下歷史新高；而聯合骨科則是在傳統旺季拉抬下，各市場出貨穩健成長，而且因為去年自4月起併入冠亞生技營收，第一季基期仍低，併購效益仍強，單月營收來到2.12億元，年增110%，單月營收亦創歷史新高。另外在醫療耗材方面，邦特因菲律賓廠接獲國際大廠客戶訂單，以及持續切入當地洗腎中心有成，元月營收來到1.31億元、大致持平前月成績，年增約28%；而血糖儀廠商泰博貴金屬試片也獲客戶採用，第一季開始拉貨，元月營收單月3.15億元，但因區域中小型客戶較多積極於年底旺季拉貨，元月營收月減約1成，但相較去年同期仍成長約28.5%。至於在隱形眼鏡方面，精華(1565)因有3條新線投產，單月營收也來到5.73元，月增4%，月增幅度較低主要係因精華報價以美元為主，元月受到台幣持續走強影響，新線貢獻受到匯率壓抑，但在出貨量增加帶動下，營收年增仍有26%。',
 None,
 1)

In [25]:
with open('bert_test.p',mode = 'wb') as pickle_file:
    pickle.dump(test, pickle_file)

#############################################################################
#########################  init_checkpoint 有重新 train ， 模型就要改  ####################

In [26]:
######## init_checkpoint 有重新 train 模型就要改
!python run_classifier.py \
  --task_name=News \
  --do_predict=true \
  --data_dir=. \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=tmp/news_output/model.ckpt-18200.index \
  --max_seq_length=512 \
  --output_dir=tmp/news_output/test_output \

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using config: {'_model_dir': 'tmp/news_output/test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd2757e88d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_l